<a href="https://colab.research.google.com/github/aymankhchman/Litllte_Project/blob/master/ideas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
import torch

def perplexity(text):
  inputs = tokenizer(text, return_tensors='pt')
  with torch.no_grad():
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
  return torch.exp(loss).item(), outputs

In [ ]:
transcript = "I need to buy something at the store."


In [ ]:
p, o = perplexity(transcript)

In [ ]:
dir(o)

In [ ]:
o.logits

In [ ]:
import torch.nn.functional as F

In [ ]:
probs = F.softmax(o.logits, dim=-1)

In [ ]:
import torch


probs[0, torch.arange(probs.shape[1]), inputs["input_ids"]]

In [ ]:
for i in range(len(inputs["input_ids"][0])):
  id = inputs["input_ids"][0][i].item()
  print(tokenizer.convert_ids_to_tokens(id))

In [ ]:
inputs["input"]

In [ ]:
o.logits.shape

In [ ]:
inputs = tokenizer('i need to buye sumthing at the stoure', return_tensors="pt", return_offsets_mapping=True)

In [ ]:
inputs["input_ids"][0][1]

In [ ]:
inputs["offset_mapping"]

In [ ]:
tokenizer.byte_encoder

In [ ]:
tokens = tokenizer.tokenize('i need to buye sumthing at the stoure.', return_tensors="pt", return_offsets_mapping=True)

In [ ]:
inputs = tokenizer('i need to buye sumthing at the stoure.', return_tensors="pt", return_offsets_mapping=True, add_special_tokens=False)

In [ ]:
inputs["input_ids"].shape

In [ ]:
len(tokens)

In [ ]:
input_ids = inputs["input_ids"]

with torch.no_grad():
    outputs = model(input_ids=input_ids, labels=input_ids)
logits = outputs.logits  # Shape: (1, seq_len, vocab_size)

In [ ]:
logits.shape

In [ ]:
# Shift logits and labels for alignment
shift_logits = logits[..., :-1, :].contiguous()  # Predictions for tokens 1...n
shift_labels = input_ids[..., 1:].contiguous()   # Ground truth for tokens 1...n

In [ ]:
log_probs = torch.nn.functional.log_softmax(shift_logits, dim=-1)


In [ ]:
log_probs_for_labels = torch.gather(log_probs, -1, shift_labels.unsqueeze(-1)).squeeze(-1)
probs = torch.exp(log_probs_for_labels).squeeze(0).tolist()

In [ ]:
probs

In [ ]:
import numpy as np

def group_tokens_into_words(tokens, probs):
    word_probs = []
    current_word = []
    current_word_log_prob = 0

    for token, prob in zip(tokens[1:], probs):  # Skip the first token (no context)
        if token.startswith("Ġ"):
            # Save previous word
            if current_word:
                word_text = "".join(current_word)
                word_prob = np.exp(current_word_log_prob)
                word_probs.append((word_text, word_prob))
            # Start new word
            current_word = [token[1:]]  # Remove "Ġ"
            current_word_log_prob = np.log(prob)
        else:
            # Append subword (e.g., "thing" to "Ġsum" → "something")
            current_word.append(token)
            current_word_log_prob += np.log(prob)

    # Add the last word
    if current_word:
        word_text = "".join(current_word)
        word_prob = np.exp(current_word_log_prob)
        word_probs.append((word_text, word_prob))

    return word_probs

word_probs = group_tokens_into_words(tokens, probs)
print("Word probabilities:", word_probs)

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch

model_name = "roberta-large"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForMaskedLM.from_pretrained(model_name)

In [ ]:
text = "I need to bye sumthing at the store."
input_ids = tokenizer.encode(text, return_tensors="pt")[0]
tokens = tokenizer.convert_ids_to_tokens(input_ids)

print("Tokens:", tokens)
# Example output: ['<s>', 'I', 'Ġneed', 'Ġto', 'Ġbye', 'Ġsum', 'thing', 'Ġat', 'Ġthe', 'Ġstore', '.', '</s>']

In [ ]:
tokens

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits  # Shape: (1, seq_len, vocab_size)

# Compute probabilities for all tokens
probs = torch.softmax(logits, dim=-1).squeeze(0)  # Shape: (seq_len, vocab_size)
token_probs = probs[torch.arange(len(input_ids)), input_ids].tolist()  # Probability of each token in context

In [ ]:
token_probs

In [ ]:
def compute_token_probabilities(text, tokenizer, model):
    input_ids = tokenizer.encode(text, return_tensors="pt")[0]
    seq_len = len(input_ids)
    token_probs = []

    for i in range(seq_len):
        # Mask the i-th token
        masked_input_ids = input_ids.clone()
        masked_input_ids[i] = tokenizer.mask_token_id

        with torch.no_grad():
            outputs = model(masked_input_ids.unsqueeze(0))

        # Get probabilities for the original token
        probs = torch.softmax(outputs.logits[0, i], dim=-1)
        original_token_id = input_ids[i].item()
        token_prob = probs[original_token_id].item()
        token_probs.append(token_prob)

    return token_probs

token_probs = compute_token_probabilities(text, tokenizer, model)
print("Token probabilities:", token_probs)

In [ ]:
def group_tokens_into_words(tokens, token_probs):
    word_probs = []
    current_word = []
    current_word_log_prob = 0

    for token, prob in zip(tokens, token_probs):
        if token.startswith("Ġ"):
            # Save previous word
            if current_word:
                word_prob = np.exp(current_word_log_prob)
                word_text = "".join(current_word)
                word_probs.append((word_text, word_prob))
            # Start new word
            current_word = [token[1:]]  # Remove "Ġ"
            current_word_log_prob = np.log(prob)
        else:
            # Append subword (e.g., "thing" to "sum")
            current_word.append(token)
            current_word_log_prob += np.log(prob)

    # Add the last word
    if current_word:
        word_prob = np.exp(current_word_log_prob)
        word_text = "".join(current_word)
        word_probs.append((word_text, word_prob))

    return word_probs

word_probs = group_tokens_into_words(tokens, token_probs)
low_prob_words = [word for word in word_probs if word[1] < 0.1]
print("Low probability words:", low_prob_words)

In [ ]:
word_probs

In [ ]:
list(zip(tokens, token_probs))

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, AutoModelForMaskedLM, AutoTokenizer
import torch
import numpy as np

# Load RoBERTa
model_name = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

text = "I need to bye sumthing at the store."

# Compute token probabilities (with special tokens handled)
def compute_token_probabilities(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"][0]
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Remove special tokens
    filtered_input_ids = input_ids[1:-1]
    filtered_tokens = tokens[1:-1]
    seq_len = len(filtered_input_ids)

    token_probs = []
    for i in range(seq_len):
        masked_input_ids = filtered_input_ids.clone()
        masked_input_ids[i] = tokenizer.mask_token_id

        # Add back special tokens for proper processing
        masked_input_ids = torch.cat([
            torch.tensor([tokenizer.bos_token_id]),
            masked_input_ids,
            torch.tensor([tokenizer.eos_token_id])
        ]).unsqueeze(0)

        with torch.no_grad():
            outputs = model(masked_input_ids)

        # Extract probability for the original token
        probs = torch.softmax(outputs.logits[0, i+1], dim=-1)
        token_prob = probs[filtered_input_ids[i]].item()
        token_probs.append(token_prob)

    return filtered_tokens, token_probs

# Group tokens into words
def group_tokens_into_words(tokens, token_probs):
    word_probs = []
    current_word = []
    current_word_log_prob = 0

    for token, prob in zip(tokens, token_probs):
        if token.startswith("Ġ"):
            if current_word:
                word_prob = np.exp(current_word_log_prob)
                word_text = "".join(current_word)
                word_probs.append((word_text, word_prob))
            current_word = [token[1:]]
            current_word_log_prob = np.log(prob)
        else:
            current_word.append(token)
            current_word_log_prob += np.log(prob)

    if current_word:
        word_prob = np.exp(current_word_log_prob)
        word_text = "".join(current_word)
        word_probs.append((word_text, word_prob))

    return word_probs

# Flag low-probability words

text = "Aye am going to bye sum bread at the supermarket"
tokens, token_probs = compute_token_probabilities(text, tokenizer, model)
word_probs = group_tokens_into_words(tokens, token_probs)
low_prob_words = [word for word in word_probs if word[1] < 1e-4]
print("Low probability words:", low_prob_words)

In [ ]:
word_probs

In [ ]:
  tokens

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_name = "FacebookAI/xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)


text = "Aye am going to bye sum bread at the supermarket"
tokens = tokenizer.tokenize(text)
print("Tokens:", tokens)

# Output: ['▁A', 'ye', '▁am', '▁going', '▁to', '▁by', 'e', '▁sum', '▁bread', '▁at', '▁the', '▁supermarket']

def group_tokens_into_words(tokens, token_probs):
    word_probs = []
    current_word = []
    current_word_log_prob = 0

    for token, prob in zip(tokens, token_probs):
        if token.startswith("▁"):
            # Save previous word
            if current_word:
                word_prob = np.exp(current_word_log_prob)
                word_text = "".join(current_word)
                word_probs.append((word_text, word_prob))
            # Start new word
            current_word = [token[1:]]  # Remove "▁"
            current_word_log_prob = np.log(prob)
        else:
            # Append subword (e.g., "ye" to "A" → "Aye")
            current_word.append(token)
            current_word_log_prob += np.log(prob)

    # Add the last word
    if current_word:
        word_prob = np.exp(current_word_log_prob)
        word_text = "".join(current_word)
        word_probs.append((word_text, word_prob))

    return word_probs

def compute_token_probabilities(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"][0]
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Remove special tokens ([CLS] and [SEP])
    filtered_input_ids = input_ids[1:-1]
    filtered_tokens = tokens[1:-1]
    seq_len = len(filtered_input_ids)

    token_probs = []
    for i in range(seq_len):
        masked_input_ids = filtered_input_ids.clone()
        masked_input_ids[i] = tokenizer.mask_token_id

        # Add back special tokens for proper processing
        masked_input_ids = torch.cat([
            torch.tensor([tokenizer.cls_token_id]),
            masked_input_ids,
            torch.tensor([tokenizer.sep_token_id])
        ]).unsqueeze(0)

        with torch.no_grad():
            outputs = model(masked_input_ids)

        # Extract probability for the original token
        probs = torch.softmax(outputs.logits[0, i+1], dim=-1)
        token_prob = probs[filtered_input_ids[i]].item()
        token_probs.append(token_prob)

    return filtered_tokens, token_probs

text = "Aye am going to bye sum bread at the supermarket"
tokens, token_probs = compute_token_probabilities(text, tokenizer, model)
word_probs = group_tokens_into_words(tokens, token_probs)
low_prob_words = [word for word in word_probs if word[1] < 1e-4]
print("Low probability words:", low_prob_words)

In [ ]:
word_probs